In [1]:
import os
import sys
import time
import glob
import datetime
import sqlite3
import numpy as np 

In [2]:
msd_subset_path='D:\millionsongsubset_full.tar\millionsongsubset_full\MillionSongSubset'
msd_subset_data_path=os.path.join(msd_subset_path,'data')
msd_subset_addf_path=os.path.join(msd_subset_path,'AdditionalFiles')
assert os.path.isdir(msd_subset_path),'wrong path' 

In [3]:
msd_code_path='D:\millionsongsubset_full.tar\millionsongsubset_full\Columbia\MSongDB'
assert os.path.isdir(msd_code_path),'wrong path' 

In [4]:
sys.path.append( os.path.join(msd_code_path,'PythonSrc') )

In [5]:
import hdf5_getters as GETTERS

In [6]:
def strtimedelta(starttime,stoptime):
    return str(datetime.timedelta(seconds=stoptime-starttime))

In [35]:
def apply_to_all_files(basedir,func=lambda x: x,ext='.h5'):
    cnt = 0
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        cnt += len(files)
        for f in files:
            func(f)       
    return cnt

In [8]:
print ('number of song files:', apply_to_all_files(msd_subset_data_path))

number of song files: 10000


In [9]:
all_artist_names = set()

In [10]:
def func_to_get_artist_name(filename):
    h5 = GETTERS.open_h5_file_read(filename)
    artist_name = GETTERS.get_artist_name(h5)
    all_artist_names.add( artist_name )
    h5.close()

In [11]:
t1 = time.time()
apply_to_all_files(msd_subset_data_path,func=func_to_get_artist_name)
t2 = time.time()
print ('all artist names extracted in:',strtimedelta(t1,t2))

all artist names extracted in: 0:02:42.698440


In [13]:
print ('found',len(all_artist_names),'unique artist names')
for k in range(5):
    print (list(all_artist_names)[k])

found 4412 unique artist names
b'Jonny Lang'
b'Cancer Bats'
b'Dimmu Borgir'
b'Kurt Tepperwein'
b'Jars Of Clay'


In [16]:
conn = sqlite3.connect(os.path.join(msd_subset_addf_path, 'subset_track_metadata.db'))
q = "SELECT DISTINCT artist_name FROM songs"
t1 = time.time()
res = conn.execute(q)
all_artist_names_sqlite = res.fetchall()
t2 = time.time()
print ('all artist names extracted (SQLite) in:',strtimedelta(t1,t2))
conn.close()

all artist names extracted (SQLite) in: 0:00:00.019922


In [17]:
for k in range(5):
    print (all_artist_names_sqlite[k][0])

!!!
(hed) p.e.
089 Clique feat. Minnesota Snipe & Skinny Cueball
089 Clique feat. Prophet
1. Futurologischer Congress


In [19]:
conn = sqlite3.connect(os.path.join(msd_subset_addf_path, 'subset_track_metadata.db'))
q = "SELECT DISTINCT artist_id FROM songs"
res = conn.execute(q)
all_artist_ids = list(map(lambda x: x[0], res.fetchall())) #Temp fix 'TypeError: 'map' object is not subscriptable' using list()
conn.close()

for k in range(4):
    print (all_artist_ids[k])

AR009211187B989185
AR00A6H1187FB5402A
AR00LNI1187FB444A5
AR00MBZ1187B9B5DB1


In [74]:
files_per_artist = {}
for aid in all_artist_ids:
    files_per_artist[aid] = 0

def func_to_count_artist_id(filename):
    h5 = GETTERS.open_h5_file_read(filename)
    artist_id = GETTERS.get_artist_id(h5)
    files_per_artist[artist_id] += 1
    h5.close()
    
# we apply this function to all files
apply_to_all_files(msd_subset_data_path,func=func_to_count_artist_id)

# the most popular artist (with the most songs) is:
most_pop_aid = sorted(files_per_artist,
                      key=files_per_artist.__getitem__,
                      reverse=True)[0]
print (most_pop_aid,'has',files_per_artist[most_pop_aid],'songs.')


KeyError: b'ARD7TVE1187B99BFB1'

In [51]:
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
q = "SELECT DISTINCT artist_name FROM songs"
q += " WHERE artist_id='"+most_pop_aid+"'"
res = conn.execute(q)
pop_artist_names = map(lambda x: x[0], res.fetchall())
conn.close()
print ('SQL query:',q)
print ('name(s) of the most popular artist:',pop_artist_names)


SQL query: SELECT DISTINCT artist_name FROM songs WHERE artist_id='AR009211187B989185'
name(s) of the most popular artist: <map object at 0x0000026770BBE208>


In [70]:
t1 = time.time()
conn = sqlite3.connect(os.path.join(msd_subset_addf_path, 'subset_track_metadata.db'))
q = "SELECT DISTINCT artist_id,artist_name,Count(track_id) FROM songs"
q += " GROUP BY artist_id"
res = conn.execute(q)
pop_artists = res.fetchall()
conn.close()
t2 = time.time()

In [71]:
print ('found most popular artist in',strtimedelta(t1,t2))

found most popular artist in 0:00:00.089718


In [68]:
print (sorted(pop_artists,key=lambda x:x[2],reverse=True)[0])

('AROIHOI122988FEB8E', 'Mario Rosenstock', 13)
